Import libs

In [2]:
from neo4j import GraphDatabase
import pathlib

Create connection

In [3]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
                
        return response
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="vovse")
relpath = pathlib.Path().absolute().as_uri()

Define ACE creation query string to avoiding pasting it multiple times 

In [36]:
ace_creation_query = """
    // ACEs - Domain groups
    FOREACH (_ IN case when ace.PrincipalType = 'DomainGroup' then [1] else [] end|
        MERGE (domainGroupACE:DomainGroup {sid:ace.PrincipalSID})
        FOREACH (_ IN case when ace.RightName = 'ExtendedRight' then [1] else [] end|
            MERGE (domainGroupACE)-[:hasExtendedRight {right:ace.AceType}]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'GenericAll' then [1] else [] end|
            MERGE (domainGroupACE)-[:hasGenericAll]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'GenericWrite' then [1] else [] end|
            MERGE (domainGroupACE)-[:hasGenericWrite]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'Owner' then [1] else [] end|
            MERGE (domainGroupACE)-[:hasOwner]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'WriteDacl' then [1] else [] end|
            MERGE (domainGroupACE)-[:hasWriteDacl]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'WriteOwner' then [1] else [] end|
            MERGE (domainGroupACE)-[:hasWriteOwner]->(principal)
        )
    )
    // ACEs - Local groups
    FOREACH (_ IN case when ace.PrincipalType = 'LocalGroup' then [1] else [] end|
        MERGE (localGroupACE:LocalGroup {sid:ace.PrincipalSID, computerName:principal.domain})
        FOREACH (_ IN case when ace.RightName = 'ExtendedRight' then [1] else [] end|
            MERGE (localGroupACE)-[:hasExtendedRight {right:ace.AceType}]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'GenericAll' then [1] else [] end|
            MERGE (localGroupACE)-[:hasGenericAll]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'GenericWrite' then [1] else [] end|
            MERGE (localGroupACE)-[:hasGenericWrite]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'Owner' then [1] else [] end|
            MERGE (localGroupACE)-[:hasOwner]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'WriteDacl' then [1] else [] end|
            MERGE (localGroupACE)-[:hasWriteDacl]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'WriteOwner' then [1] else [] end|
            MERGE (localGroupACE)-[:hasWriteOwner]->(principal)
        )
    )
    // ACEs - Domain users
    FOREACH (_ IN case when ace.PrincipalType = 'DomainUser' then [1] else [] end|
        MERGE (domainUserACE:DomainUser {sid:ace.PrincipalSID})
        FOREACH (_ IN case when ace.RightName = 'ExtendedRight' then [1] else [] end|
            MERGE (domainUserACE)-[:hasExtendedRight {right:ace.AceType}]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'GenericAll' then [1] else [] end|
            MERGE (domainUserACE)-[:hasGenericAll]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'GenericWrite' then [1] else [] end|
            MERGE (domainUserACE)-[:hasGenericWrite]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'Owner' then [1] else [] end|
            MERGE (domainUserACE)-[:hasOwner]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'WriteDacl' then [1] else [] end|
            MERGE (domainUserACE)-[:hasWriteDacl]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'WriteOwner' then [1] else [] end|
            MERGE (domainUserACE)-[:hasWriteOwner]->(principal)
        )
    )
    // ACEs - Computers
    FOREACH (_ IN case when ace.PrincipalType = 'Computer' then [1] else [] end|
        MERGE (computerACE:Computer {sid:ace.PrincipalSID})
        FOREACH (_ IN case when ace.RightName = 'ExtendedRight' then [1] else [] end|
            MERGE (computerACE)-[:hasExtendedRight {right:ace.AceType}]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'GenericAll' then [1] else [] end|
            MERGE (computerACE)-[:hasGenericAll]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'GenericWrite' then [1] else [] end|
            MERGE (computerACE)-[:hasGenericWrite]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'Owner' then [1] else [] end|
            MERGE (computerACE)-[:hasOwner]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'WriteDacl' then [1] else [] end|
            MERGE (computerACE)-[:hasWriteDacl]->(principal)
        )
        FOREACH (_ IN case when ace.RightName = 'WriteOwner' then [1] else [] end|
            MERGE (computerACE)-[:hasWriteOwner]->(principal)
        )
    )
    """

Import data

In [39]:
# Delete everything in graph
query = '''MATCH (n) DETACH DELETE n'''
out = conn.query(query)

# User data
query = '''
CALL apoc.load.json('%s/../jsondata/0001-users.json') YIELD value
UNWIND value.users AS u
MERGE (domainUser:DomainUser {sid:u.SID})
SET domainUser += u.Properties

// ACEs
WITH u, domainUser AS principal
UNWIND u.Aces AS ace
%s
''' % (relpath, ace_creation_query)
out = conn.query(query)

# Group data - DC local groups
query = '''
CALL apoc.load.json('%s/../jsondata/0001-groups.json') YIELD value
UNWIND value.groups AS g
WITH g WHERE g.type = 'LocalGroup'
MERGE (localGroup:LocalGroup {sid:g.SID, computerName:g.Properties.domain})
SET localGroup += g.Properties
WITH g, localGroup
UNWIND g.members AS mb
FOREACH (_ IN case when mb.type = 'LocalGroup' then [1] else [] end|
	MERGE (localSubGroup:LocalGroup {sid:mb.SID, computerName:g.Properties.domain})
	MERGE (localGroup)<-[:memberOf]-(localSubGroup)
)
FOREACH (_ IN case when mb.type = 'DomainUser' then [1] else [] end|
	MERGE (domainUser:DomainUser {sid:mb.SID})
	MERGE (localGroup)<-[:memberOf]-(domainUser)
)
FOREACH (_ IN case when mb.type = 'DomainGroup' then [1] else [] end|
	MERGE (domainGroup:DomainGroup {sid:mb.SID})
	MERGE (localGroup)<-[:memberOf]-(domainGroup)
)

// ACEs
WITH g, localGroup AS principal
UNWIND g.Aces AS ace
%s
''' % (relpath, ace_creation_query)
out = conn.query(query)

# Group data - domain groups
query = '''
CALL apoc.load.json('%s/../jsondata/0001-groups.json') YIELD value
UNWIND value.groups AS g
WITH g WHERE g.type = 'DomainGroup'
MERGE (domainGroup:DomainGroup {sid:g.SID})
SET domainGroup += g.Properties
WITH g, domainGroup
UNWIND g.members AS mb
FOREACH (_ IN case when mb.type = 'LocalGroup' then [1] else [] end|
    MERGE (localGroup:LocalGroup {sid:mb.SID, computerName:g.Properties.domain})
    MERGE (domainGroup)<-[:memberOf]-(localGroup)
)
FOREACH (_ IN case when mb.type = 'DomainUser' then [1] else [] end|
    MERGE (domainUser:DomainUser {sid:mb.SID})
    MERGE (domainGroup)<-[:memberOf]-(domainUser)
)
FOREACH (_ IN case when mb.type = 'DomainGroup' then [1] else [] end|
    MERGE (domainSubGroup:DomainGroup {sid:mb.SID})
    MERGE (domainGroup)<-[:memberOf]-(domainSubGroup)
)

// ACEs
WITH g, domainGroup AS principal
UNWIND g.Aces AS ace
%s
''' % (relpath, ace_creation_query)
out = conn.query(query)

# Computer data
query = '''
CALL apoc.load.json('%s/../jsondata/0001-computers.json') YIELD value
UNWIND value.computers AS c
MERGE (computer:Computer {sid:c.SID})
SET computer += c.Properties

// Local users
WITH c, computer
UNWIND c.LocalUsers AS lu
MERGE (localUser:LocalUser {name:lu.name, sid:lu.SID, computerName:computer.name})
MERGE (computer)-[:hasUser]->(localUser)

// Local groups
WITH c, computer
UNWIND c.LocalGroups AS lg
MERGE (localGroup:LocalGroup {name:lg.name, sid:lg.SID, computerName:computer.name})
MERGE (computer)-[:hasGroup]->(localGroup)

// Local group members
WITH lg, localGroup, c, computer
UNWIND lg.members AS mb
FOREACH (_ IN case when mb.type = 'LocalGroup' then [1] else [] end|
	MERGE (localSubGroup:LocalGroup {name:mb.name, sid:mb.SID, computerName:computer.name})
	MERGE (computer)-[:hasGroup]->(localSubGroup)
	MERGE (localGroup)<-[:memberOf]-(localSubGroup)
)
FOREACH (_ IN case when mb.type = 'LocalUser' then [1] else [] end|
	MERGE (localUser:LocalUser {name:mb.name, sid:mb.SID, computerName:computer.name})
	MERGE (computer)-[:hasUser]->(localUser)
	MERGE (localGroup)<-[:memberOf]-(localUser)
)
FOREACH (_ IN case when mb.type = 'DomainUser' then [1] else [] end|
	MERGE (domainUser:DomainUser {sid:mb.SID})
	MERGE (localGroup)<-[:memberOf]-(domainUser)
)
FOREACH (_ IN case when mb.type = 'DomainGroup' then [1] else [] end|
	MERGE (domainGroup:DomainGroup {sid:mb.SID})
	MERGE (localGroup)<-[:memberOf]-(domainGroup)
)

// URA
WITH c, computer
UNWIND c.URA AS ura
FOREACH (principal IN ura.principals |
    FOREACH (_ IN case when principal.type = 'LocalGroup' then [1] else [] end|
        MERGE (localGroup:LocalGroup {name:principal.name, sid:principal.SID, computerName:computer.name})
        MERGE (localGroup)-[:hasRight {name:ura.right}]->(computer)
    )
    FOREACH (_ IN case when principal.type = 'LocalUser' then [1] else [] end|
        MERGE (localUser:LocalUser {name:principal.name, sid:principal.SID, computerName:computer.name})
        MERGE (localUser)-[:hasRight {name:ura.right}]->(computer)
    )
    FOREACH (_ IN case when principal.type = 'DomainUser' then [1] else [] end|
        MERGE (domainUser:DomainUser {sid:principal.SID})
        MERGE (domainUser)-[:hasRight {name:ura.right}]->(computer)
    )
    FOREACH (_ IN case when principal.type = 'DomainGroup' then [1] else [] end|
        MERGE (domainGroup:DomainGroup {sid:principal.SID})
        MERGE (domainGroup)-[:hasRight {name:ura.right}]->(computer)
    )
)

// ACEs
WITH c, computer AS principal
UNWIND c.Aces AS ace
%s
''' % (relpath, ace_creation_query)
out = conn.query(query)

# Container data
query = '''
CALL apoc.load.json('%s/../jsondata/0001-containers.json') YIELD value
UNWIND value.containers AS c
CREATE (container:Container {GUID:c.ObjectGUID})
SET container += c.Properties
WITH c, container
UNWIND c.Principals AS principal
FOREACH (_ IN case when principal.type = 'LocalUser' then [1] else [] end|
	MERGE (localUser:LocalUser {sid:principal.SID, computerName:c.Properties.domain})
	MERGE (container)-[:contains]->(localUser)
)
FOREACH (_ IN case when principal.type = 'LocalGroup' then [1] else [] end|
	MERGE (localGroup:LocalGroup {sid:principal.SID, computerName:c.Properties.domain})
	MERGE (container)-[:contains]->(localGroup)
)
FOREACH (_ IN case when principal.type = 'DomainUser' then [1] else [] end|
	MERGE (domainUser:DomainUser {sid:principal.SID})
	MERGE (container)-[:contains]->(domainUser)
)
FOREACH (_ IN case when principal.type = 'DomainGroup' then [1] else [] end|
	MERGE (domainGroup:DomainGroup {sid:principal.SID})
	MERGE (container)-[:contains]->(domainGroup)
)
FOREACH (_ IN case when principal.type = 'Computer' then [1] else [] end|
	MERGE (computer:Computer {sid:principal.SID})
	MERGE (container)-[:contains]->(computer)
)

// Sub containers
WITH c, container
UNWIND c.SubContainers AS sc
MERGE (subContainer:Container {GUID:sc})
MERGE (container)-[:contains]->(subContainer)

// ACEs
WITH c, container AS principal
UNWIND c.Aces AS ace
%s
''' % (relpath, ace_creation_query)
out = conn.query(query)

# Connect DC groups to DCs
query = '''
MATCH (dc:Computer) WHERE dc.PrimaryGroupSid ENDS WITH '-516'
WITH dc
MATCH (dcGroup:LocalGroup) WHERE dcGroup.domain = dc.domain
MERGE (dc)-[:hasGroup]->(dcGroup)
'''
out = conn.query(query)

Get all data

In [15]:
query = '''MATCH (n) RETURN n'''
out = conn.query(query)

Get domain users and groups which are admin on a host

In [46]:
query = '''
MATCH (adminGrp:LocalGroup)
WHERE adminGrp.sid = 'S-1-5-32-544'
WITH adminGrp
MATCH (adminGrp)-[:hasMember*1..]->(x)
WHERE x:DomainUser OR x:DomainGroup
RETURN DISTINCT x.name AS SamAccountName, adminGrp.computerName AS computer
'''
out = conn.query(query)
out

[<Record SamAccountName='ENTERPRISE ADMINS@HOT.LOCAL' computer='HOT.LOCAL'>,
 <Record SamAccountName='ADMINISTRATOR@HOT.LOCAL' computer='HOT.LOCAL'>,
 <Record SamAccountName='DOMAIN ADMINS@HOT.LOCAL' computer='HOT.LOCAL'>,
 <Record SamAccountName='MJ@HOT.LOCAL' computer='HOT.LOCAL'>,
 <Record SamAccountName='KR@HOT.LOCAL' computer='HOT.LOCAL'>,
 <Record SamAccountName='TESTBJ@HOT.LOCAL' computer='HOT.LOCAL'>,
 <Record SamAccountName='T1_BJ@HOT.LOCAL' computer='HOT.LOCAL'>,
 <Record SamAccountName='BENNY@HOT.LOCAL' computer='HOT.LOCAL'>,
 <Record SamAccountName='DOMAIN-ADMIN@HOT.LOCAL' computer='HOT.LOCAL'>,
 <Record SamAccountName='DOMAIN USERS@HOT.LOCAL' computer='HOT-JUMPHOST.HOT.LOCAL'>]

Get containers

In [47]:
query = '''
MATCH (container:Container)
RETURN container.name, container.GUID
'''
out = conn.query(query)
out

[<Record container.name='COMPUTERS@HOT.LOCAL' container.GUID='BBFB75BA-0F23-4CFD-BB2F-24CE2FBA907A'>,
 <Record container.name='BUILTIN@HOT.LOCAL' container.GUID='1F1A4CFC-C0B3-4B06-AE85-F78E98B4CCA8'>,
 <Record container.name='USERS@HOT.LOCAL' container.GUID='75A39C0B-0B01-4595-ABFC-0E1A993B1AD8'>,
 <Record container.name='JUMPHOST@HOT.LOCAL' container.GUID='01A0984A-DD08-44E2-A6BF-D5D7AEAF02BF'>,
 <Record container.name='DOMAIN CONTROLLERS@HOT.LOCAL' container.GUID='98766D6D-A3F5-4D0A-8AAC-DED1AE48FB2C'>,
 <Record container.name='WS@HOT.LOCAL' container.GUID='5A76B49C-953E-449A-A8F0-31974CD2F7F2'>]

Define tiers

In [40]:
query = '''
CALL apoc.load.json('%s/../jsondata/0001-tiering-containers.json') YIELD value
UNWIND value.containers AS c
MERGE (tier:Tier {level:c.tier})
WITH tier, c
MATCH (container:Container {GUID:c.GUID})
MATCH (container)-[:contains]->(principal) WHERE (principal:LocalUser) OR (principal:DomainUser) OR (principal:Computer)
MERGE (principal)-[:inTier]->(tier)
''' % relpath
out = conn.query(query)

Get all 2. order paths from Tier 2 to Tier 1

In [ ]:
MATCH path=(:Tier {level:"1"})<-[:inTier]-()-[*1..2]-()-[:inTier]->(:Tier {level:"2"}) RETURN path